In [1]:

import numpy as np
import pandas as pd

import pyspark


from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


#################################             FUNCTIONNALITY             ########################################


# this programs reads a csv file from the TBF110. 

# The program counts the sum ITEMS_TOTAL_AMOUNT for each sender_id and each year

# It creates 0 values for the non existing dates for each sender_id and stores it in a csv for later visualizing


#################################################################################################################



# Set up the spark session

spark = SparkSession.builder \
   .master("spark://spark-master:7077") \
   .appName("sumbyyears") \
   .config(conf = pyspark.SparkConf()) \
   .getOrCreate()



# Read the csv File

df1 = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('../sparkJup/romain.csv')



dftest = df1.groupby("year","SENDER_SITE_LOC_ID").agg(sum('ITEMS_TOTAL_AMOUNT'))
dftest.show()

dfYear = dftest.toPandas()



i = 0


# Remove the duplicates for sender_id

senderId = pd.unique(dfYear['SENDER_SITE_LOC_ID'].values)


# A table with all the dates which occur one time

dateByYear = pd.unique(dfYear['year'].values)


# Set up the table length

lenDateByYear = len(dateByYear)
lenSenderId = len(senderId)




# This loop is used to create some rows for sender_id which did not have all the dates
# Example sender_id = 1 year = 2017 dateByYear=[2017,2018]
# A new row is created to have sender_id = 1 year 2017 and sender_id = 1 year = 2018
# This is compulsory for creating visualization

while i < lenSenderId:
    for j in range(lenDateByYear):
        if i+j < lenSenderId + lenDateByYear:
            dftemp = dfYear.loc[dfYear['SENDER_SITE_LOC_ID'].isin([senderId[i]]) & dfYear['year'].isin([dateByYear[j]])]
            #print(senderId[i])
            #print(dateByYear[j])
            if (dftemp.empty == True):
                df3 = pd.DataFrame(np.array([[dateByYear[j],senderId[i]]]), columns=['year','SENDER_SITE_LOC_ID'])
                dfYear = dfYear.append(df3, ignore_index=True)
#                #print(dfsum['SENDER_SITE_LOC_ID'][i])
#                #print(dfsum['year'][i])
#                print(tab2[j])
#                print('not ok')
            else :
                print('ok')
                #print(dfsum['SENDER_SITE_LOC_ID'][i])
                #print(dfsum['year'][i])
                #print(tab2[j])


        else:
            if(dfsum['year'][i+j - 1] != dateByYear[len(dateByYear)-1]):
                #print('make something')
                df3 = pd.DataFrame(np.array([[dateByYear[j],senderId[i]]]), columns=['year','SENDER_SITE_LOC_ID'])
                dfYear = dfYear.append(df3, ignore_index=True)
    i = i + 1


print(dfYear)


# Replace the values which are NaN by 0

dfYear= dfYear.fillna(0)

print(dfYear)


# Save to a csv File

dfYear.to_csv('sumbyyears.csv')


+----+------------------+-----------------------+
|year|SENDER_SITE_LOC_ID|sum(ITEMS_TOTAL_AMOUNT)|
+----+------------------+-----------------------+
|2017|             15705|             1059237974|
|2017|             15801|            12531848256|
|2017|              8627|            28521692304|
|2017|             15793|           748974778302|
|2017|              7597|              524884448|
|2018|              7597|             2254851640|
|2017|             15695|               12979272|
+----+------------------+-----------------------+

ok
ok
ok
ok
ok
ok
ok
    SENDER_SITE_LOC_ID  sum(ITEMS_TOTAL_AMOUNT)  year
0                15705             1.059238e+09  2017
1                15801             1.253185e+10  2017
2                 8627             2.852169e+10  2017
3                15793             7.489748e+11  2017
4                 7597             5.248844e+08  2017
5                 7597             2.254852e+09  2018
6                15695             1.297927e+07  2